In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
sys.path.append('../input/birdclefmodels/')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import librosa

from matplotlib import pyplot as plt
import json
import random
from tqdm import tqdm
import os
import yaml

import torch
from torch import nn
import torchaudio as ta
import timm
from torch.utils.data import Dataset, DataLoader

import neural_network

In [ ]:
data_root = '/kaggle/input/birdclef-2022'
train_meta = pd.read_csv('../input/birdclef-data-with-wav-durations/train_metadata_extended.csv')
ebird_taxonomy = pd.read_csv(os.path.join(data_root, 'eBird_Taxonomy_v2021.csv'))

In [ ]:
train_meta.loc[:, 'secondary_labels'] = train_meta.secondary_labels.apply(eval)
train_meta['target_raw'] = train_meta.secondary_labels + train_meta.primary_label.apply(lambda x: [x])

In [ ]:
all_species = sorted(set(train_meta.target_raw.sum()))
species2id = {s: i for i, s in enumerate(all_species)}
id2species = {i: s for i, s in enumerate(all_species)}

train_meta['target'] = train_meta.target_raw.apply(lambda species: [int(s in species) for s in all_species])

In [ ]:
def load_wav(fname, offset, duration):
#     fname = 'afrsil1/XC125458.ogg'
    fpath = os.path.join(data_root, 'train_audio', fname)
    wav, sr = librosa.load(fpath, sr=None, duration=duration)
    assert sr <= 32000, sr
    return wav, sr

### Model load

In [1]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def init_model(config_path, model_path):
    with open(config_path) as fin:
        config = yaml.safe_load(fin)

    model_config = config['model']
    if 'backbone_config' in model_config['params']:
        model_config['params']['backbone_config']['pretrained'] = False
    else:
        model_config['params']['backbone_config'] = {'pretrained': False}
    data_config = config['data']
    model_class = NN_CATALOG[model_config['name']]

    model = model_class(len(all_species), int(data_config['crop_len'] // data_config['test_wav_len']),
                        **model_config['params'])
    model.to(device)

    state_dict = torch.load(model_path, map_location=device)
    model.load_state_dict(state_dict)
    return model


In [ ]:
from neural_network import NN_CATALOG

ckpt_root = '../input/birdclefsubmit'
eff_path = ('eff/baseline_config.yaml', 'eff/final-model.pt')
baseline_path = ('baseline/baseline_config.yaml', 'baseline/final-model.pt')

models = [init_model(*p) for p in [eff_path, baseline_path]]
weights = None


In [ ]:
class Blending(nn.Module):
    def __init__(self, models, weights):
        self.models = models
        self.weights = weights or [1 / len(models) for _ in range(models)]
        
    def forward(self, wav_tensor):
        pred = [m(wav_tensor)['logits'] for m in self.models]
        pred = sum(p * w for p, w in zip(pred, self.weights))
        return pred

### Torch Dataset

In [ ]:
with open(os.path.join(data_root, 'scored_birds.json')) as fin:
    test_birds = json.load(fin)

In [ ]:
class Compose:
    def __init__(self, transforms: list):
        self.transforms = transforms

    def __call__(self, y: np.ndarray, sr):
        for trns in self.transforms:
            y = trns(y, sr)
        return y
    
    
class AudioTransform:
    def __init__(self, always_apply=False, p=0.5):
        self.always_apply = always_apply
        self.p = p

    def __call__(self, y: np.ndarray, sr):
        if self.always_apply:
            return self.apply(y, sr=sr)
        else:
            if np.random.rand() < self.p:
                return self.apply(y, sr=sr)
            else:
                return y

    def apply(self, y: np.ndarray, **params):
        raise NotImplementedError
        
        
class Normalize(AudioTransform):
    def __init__(self, always_apply=False, p=1):
        super().__init__(always_apply, p)

    def apply(self, y: np.ndarray, **params):
        max_vol = np.abs(y).max()
        y_vol = y / max_vol
        assert not np.isnan(y_vol).any(), f'{max_vol}'
        return y_vol

In [ ]:
class TestDataset(Dataset):
    def __init__(self, test_folder, augmentations=None):
        super().__init__()
        self.test_folder = test_folder
        self.fnames = [f for f in os.listdir(test_folder) if f.endswith('.ogg')]
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.fnames)
    
    def __getitem__(self, idx):
        fpath = os.path.join(self.test_folder, self.fnames[idx])
        wav, sr = load_wav(fpath, 0, None)
        if self.augmentations:
            wav = self.augmentations(wav, None)
        wav = torch.tensor(wav)
        assert (13 * 5 * sr) > len(wav) 
        wav = wav[:len(wav) // 12 * 12].reshape((12, len(wav) // 12))
        return wav

In [ ]:
test_dataset = TestDataset(
    os.path.join(data_root, 'test_soundscapes'),
    Compose([Normalize(p=1)])
)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=2,
    drop_last=False,   
)

In [ ]:
def find_topk(pred_proba, max_birds=5):
    pred_proba = pred_proba[:, [species2id[b] for b in test_birds]]
    mean_proba = pred_proba.mean(axis=0)
    topk_birds = [i for i, _ in sorted(enumerate(mean_proba),
                                       key=lambda x: x[1],
                                       reverse=True)][:max_birds]
    return topk_birds

In [ ]:
treshold_dict = {'akiapo': 0.1,
 'aniani': 0.05,
 'apapan': 0.05,
 'barpet': 0.05,
 'crehon': 0.05,
 'elepai': 0.05,
 'ercfra': 0.01,
 'hawama': 0.05,
 'hawcre': 0.05,
 'hawgoo': 0.05,
 'hawhaw': 0.01,
 'hawpet1': 0.05,
 'houfin': 0.1,
 'iiwi': 0.2,
 'jabwar': 0.05,
 'maupar': 0.05,
 'omao': 0.05,
 'puaioh': 0.05,
 'skylar': 0.1,
 'warwhe1': 0.05,
 'yefcan': 0.15}

In [ ]:
pred_list = []
treshold = 0.1
model.eval()
with torch.no_grad():
    for i, batch in tqdm(enumerate(test_dataloader)):
        batch_size, part_count, part_size = batch.shape
        batch = batch.reshape(batch_size * part_count, part_size)
        pred = model(batch.to(device))['logits']
        pred = pred.cpu().numpy()
#         topk_birds = find_topk(pred_proba, max_birds=10)
        
        for j, chunk_pred in enumerate(pred):
            inbatch_number = j // part_count
            chunk_number = j % part_count + 1
            f_idx = i * batch_size + inbatch_number
            fname = test_dataset.fnames[f_idx]
            prefix = fname.split('.')[0]
            sufix = f'{5 * chunk_number}'
            
            pred_list.extend([{
                'row_id': '_'.join([prefix, b, sufix]),
                'target': chunk_pred[species2id[b]] > treshold_dict[b] # if species2id[b] in topk_birds else False
            } for b in test_birds])
pred_pd = pd.DataFrame(pred_list)

In [ ]:
pred_pd.to_csv("submission.csv", index=False)
pred_pd